# QEPC – Player Lambdas (Recency + vs Opponent) – Upgraded

This notebook builds **roster-aware** player-level λ (lambda) projections for **points, rebounds, assists** using:

- Only games **before** the target game date (no data leakage).
- A **lookback window** (default 2 seasons) to avoid ancient-history noise.
- **Roster detection** (from game_id if available; otherwise inferred from recent games).
- A **share model** (team-total × player share) with recency + vs-opp adjustments (shrunk to prevent tiny-sample madness).
- Optional fallback logic for players with little/no history.

Numbered cells below are designed to be copy/paste safe.


In [ ]:
# ==========================================================
# CELL 1 – PROJECT ROOT AUTO-DETECT + IMPORTS
# ==========================================================
from __future__ import annotations

import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd

def find_project_root(start: Path | None = None, package_name: str = "qepc") -> Path | None:
    r"""
    Find the QEPC project root by walking upward from CWD and checking common locations.
    Works across machines (different Windows usernames) without hardcoding C:/Users/<name>/...
    """
    start = (start or Path.cwd()).resolve()

    # 1) Walk up from current directory
    for p in [start, *start.parents]:
        if (p / package_name / "__init__.py").exists():
            return p
        if p.name.lower() == "qepc_project" and (p / package_name).exists():
            return p

    # 2) Environment override (optional)
    env = os.getenv("QEPC_PROJECT_ROOT")
    if env:
        cand = Path(env).expanduser().resolve()
        if (cand / package_name / "__init__.py").exists():
            return cand

    # 3) Common spots under home directory
    home = Path.home()
    for cand in [
        home / "qepc_project",
        home / "Documents" / "qepc_project",
        home / "Desktop" / "qepc_project",
    ]:
        if (cand / package_name / "__init__.py").exists():
            return cand

    return None

PROJECT_ROOT = find_project_root()

if PROJECT_ROOT is None:
    raise FileNotFoundError(
        "Could not auto-detect your QEPC project root.\n"
        "Fix options:\n"
        "  1) Open Jupyter with working directory inside your repo\n"
        "  2) Or set env var QEPC_PROJECT_ROOT to the repo path\n"
        "     e.g. PowerShell:  $env:QEPC_PROJECT_ROOT = 'C:/Users/YOU/qepc_project'\n"
    )

# Make sure Python can import `qepc`
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("✅ PROJECT_ROOT:", PROJECT_ROOT)

import qepc
print("✅ qepc package:", Path(qepc.__file__).resolve())

CACHE_IMPORTS = PROJECT_ROOT / "cache" / "imports"
print("📦 CACHE_IMPORTS:", CACHE_IMPORTS)


In [ ]:
# ==========================================================
# CELL 2 – LOAD QEPC-READY EOIN DATA (WITH FALLBACKS)
# ==========================================================
from pathlib import Path

def _coerce_game_date(df: pd.DataFrame, col: str = "game_date") -> pd.DataFrame:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce").dt.date
    return df

def _fallback_load_parquet(name: str) -> pd.DataFrame:
    path = CACHE_IMPORTS / name
    if not path.exists():
        raise FileNotFoundError(f"Missing cache file: {path}")
    return pd.read_parquet(path)

# Try QEPC loaders first (preferred)
try:
    from qepc.nba.eoin_data_source import (
        load_eoin_games,
        load_eoin_player_boxes,
        load_eoin_team_boxes,
        print_eoin_summary,
    )

    games_qepc = load_eoin_games()
    player_boxes_qepc = load_eoin_player_boxes()
    team_boxes_qepc = load_eoin_team_boxes()

    print_eoin_summary(games_qepc, player_boxes_qepc, team_boxes_qepc)

except Exception as e:
    print("⚠️ QEPC loader import failed; falling back to cached parquet in cache/imports/")
    print("   Error:", repr(e))

    games_qepc = _fallback_load_parquet("games_qepc.parquet")
    player_boxes_qepc = _fallback_load_parquet("player_boxes_qepc.parquet")
    team_boxes_qepc = _fallback_load_parquet("team_boxes_qepc.parquet")

# Normalize date columns
games_qepc = _coerce_game_date(games_qepc, "game_date")
player_boxes_qepc = _coerce_game_date(player_boxes_qepc, "game_date")
team_boxes_qepc = _coerce_game_date(team_boxes_qepc, "game_date")

print("\nShapes:")
print("  games_qepc:", games_qepc.shape)
print("  player_boxes_qepc:", player_boxes_qepc.shape)
print("  team_boxes_qepc:", team_boxes_qepc.shape)

# Quick schema sanity checks (non-fatal)
for col in ["game_id", "team_name", "opp_team_name", "player_id", "points", "reboundstotal", "assists", "numminutes"]:
    if col not in player_boxes_qepc.columns:
        print(f"⚠️ player_boxes_qepc missing expected column: {col}")


In [ ]:
# ==========================================================
# CELL 3 – PICK A GAME (BY DATE + OPTIONAL TEAMS) + GET TEAM TOTAL TARGETS
# ==========================================================
import datetime as _dt

# ----- USER CONTROLS -----
GAME_DATE = "2025-12-05"        # YYYY-MM-DD
HOME_TEAM = "Celtics"           # or None
AWAY_TEAM = "Lakers"            # or None

# If your matchup engine exists, use it (best).
# Otherwise we fall back to a minimal table built from games_qepc.
try:
    from qepc.nba.matchups_eoin import build_matchups_for_date
    matchups = build_matchups_for_date(GAME_DATE)
    print("✅ Loaded matchups via qepc.nba.matchups_eoin.build_matchups_for_date")
except Exception as e:
    print("⚠️ Could not import matchup engine; using games_qepc as fallback.")
    print("   Error:", repr(e))
    gd = pd.to_datetime(GAME_DATE).date()
    cols = [c for c in ["game_id","game_date","home_team_name","away_team_name","home_team_id","away_team_id"] if c in games_qepc.columns]
    matchups = games_qepc[games_qepc["game_date"] == gd][cols].copy()
    # fallback team totals if matchup engine missing
    matchups["exp_home_pts"] = np.nan
    matchups["exp_away_pts"] = np.nan

print("matchups.shape:", matchups.shape)
display(matchups.head(10))

# Pick the game row
if HOME_TEAM and AWAY_TEAM:
    sel = matchups[
        (matchups["home_team_name"].astype(str) == str(HOME_TEAM))
        & (matchups["away_team_name"].astype(str) == str(AWAY_TEAM))
    ]
    if len(sel) == 0:
        raise ValueError(f"No matchup found for {GAME_DATE} with {HOME_TEAM} vs {AWAY_TEAM}.")
    game_row = sel.iloc[0]
else:
    if len(matchups) == 0:
        raise ValueError(f"No games found for date {GAME_DATE}.")
    game_row = matchups.iloc[0]

home_team = str(game_row["home_team_name"])
away_team = str(game_row["away_team_name"])
asof_date = pd.to_datetime(game_row["game_date"]).date()
game_id = int(game_row["game_id"]) if "game_id" in game_row and pd.notna(game_row["game_id"]) else None

print("\n🎯 Selected game:")
print("  game_id:", game_id)
print("  date:", asof_date)
print("  matchup:", f"{away_team} @ {home_team}")

home_pts_target = float(game_row.get("exp_home_pts", np.nan))
away_pts_target = float(game_row.get("exp_away_pts", np.nan))

print("\nTeam point targets from matchup engine (NaN means fallback/unknown):")
print("  home_pts_target:", home_pts_target)
print("  away_pts_target:", away_pts_target)


In [ ]:
# ==========================================================
# CELL 4 – CONFIG: LOOKBACK WINDOW + MODEL WEIGHTS (NO DATA LEAKAGE)
# ==========================================================
from datetime import timedelta

# How much history to use before the target game date
LOOKBACK_DAYS = 365 * 2        # ~2 seasons; increase if you want more stability, decrease if you want more recency
LAST_N_GAMES = 10              # player recency window (per player)
MIN_GAMES_PLAYER = 5           # minimum games to trust season averages
MIN_GAMES_VS_OPP = 3           # minimum games vs opponent to even consider it
VSOPP_SHRINK_K = 8             # bigger = more shrinkage toward season avg when vs-opp sample is small

# Share-model adjustment strengths
W_RECENCY = 0.60               # how much recency nudges shares (0..1)
W_VSOPP   = 0.40               # how much matchup history nudges shares (0..1)

# Clip factors to avoid nuclear weirdness from tiny samples / injuries / etc.
FACTOR_CLIP = (0.60, 1.60)

# If True, we build points via TEAM_TOTAL × ADJUSTED_SHARE (recommended).
# If False, we build points from blended per-player averages then scale to team total.
USE_SHARE_MODEL = True

print("Config loaded.")
print("  LOOKBACK_DAYS:", LOOKBACK_DAYS)
print("  LAST_N_GAMES:", LAST_N_GAMES)
print("  USE_SHARE_MODEL:", USE_SHARE_MODEL)


In [ ]:
# ==========================================================
# CELL 5 – BUILD HISTORY SLICE + GET ROSTERS (GAME_ID OR RECENT INFERENCE)
# ==========================================================
def slice_player_history(player_boxes: pd.DataFrame, asof_date: _dt.date, lookback_days: int) -> pd.DataFrame:
    pb = player_boxes.copy()
    pb["game_date"] = pd.to_datetime(pb["game_date"], errors="coerce").dt.date
    pb = pb[pb["game_date"].notna()]
    pb = pb[pb["game_date"] < asof_date]  # STRICTLY BEFORE (no leakage)
    if lookback_days is not None and lookback_days > 0:
        cutoff = asof_date - timedelta(days=int(lookback_days))
        pb = pb[pb["game_date"] >= cutoff]
    return pb

pb_hist = slice_player_history(player_boxes_qepc, asof_date=asof_date, lookback_days=LOOKBACK_DAYS)
print("pb_hist rows:", len(pb_hist), "| date window:", (pb_hist["game_date"].min(), pb_hist["game_date"].max()))

def roster_from_game_id(player_boxes: pd.DataFrame, game_id: int, team_name: str) -> list[int]:
    gp = player_boxes[player_boxes["game_id"].astype(int) == int(game_id)]
    ids = gp.loc[gp["team_name"].astype(str) == str(team_name), "player_id"].unique()
    return sorted(int(x) for x in ids)

def roster_from_recent_games(player_boxes_hist: pd.DataFrame, team_name: str, lookback_games: int = 6) -> list[int]:
    team_rows = player_boxes_hist[player_boxes_hist["team_name"].astype(str) == str(team_name)].copy()
    if len(team_rows) == 0:
        return []
    # last N unique games for that team (in the history slice)
    gids = (
        team_rows.sort_values("game_date")["game_id"]
        .dropna()
        .astype(int)
        .drop_duplicates()
        .tail(int(lookback_games))
        .tolist()
    )
    ids = team_rows[team_rows["game_id"].astype(int).isin(gids)]["player_id"].unique()
    return sorted(int(x) for x in ids)

def get_roster_ids(game_id: int | None, team_name: str) -> list[int]:
    if game_id is not None:
        ids = roster_from_game_id(player_boxes_qepc, game_id, team_name)
        if len(ids) > 0:
            return ids
    # fallback (works for "future" games where we don't have a game_id roster)
    return roster_from_recent_games(pb_hist, team_name, lookback_games=6)

home_ids = get_roster_ids(game_id, home_team)
away_ids = get_roster_ids(game_id, away_team)

print(f"Roster for {home_team}: {len(home_ids)} players")
print("  home_ids:", home_ids)
print(f"Roster for {away_team}: {len(away_ids)} players")
print("  away_ids:", away_ids)

if len(home_ids) == 0 or len(away_ids) == 0:
    print("⚠️ One roster is empty. If this is a future game, increase LOOKBACK_DAYS or make sure pb_hist has recent games.")


In [ ]:
# ==========================================================
# CELL 6 – BUILD ROSTER USAGE TABLES + VS-OPP SPLITS (POINTS/REB/AST)
# ==========================================================
def _safe_name_from_rows(rows: pd.DataFrame, pid: int) -> str:
    if "firstname" in rows.columns and "lastname" in rows.columns and len(rows) > 0:
        first = str(rows["firstname"].iloc[-1]) if pd.notna(rows["firstname"].iloc[-1]) else ""
        last  = str(rows["lastname"].iloc[-1]) if pd.notna(rows["lastname"].iloc[-1]) else ""
        nm = (first + " " + last).strip()
        if nm and nm.lower() != "nan":
            return nm
    return f"player_{pid}"

def _add_team_totals(pb_team: pd.DataFrame) -> pd.DataFrame:
    totals = (
        pb_team.groupby(["team_name","game_id"], as_index=False)
        .agg(
            team_pts=("points","sum"),
            team_reb=("reboundstotal","sum"),
            team_ast=("assists","sum"),
        )
    )
    out = pb_team.merge(totals, on=["team_name","game_id"], how="left")
    for num, den, outcol in [
        ("points","team_pts","points_share"),
        ("reboundstotal","team_reb","reb_share"),
        ("assists","team_ast","ast_share"),
    ]:
        out[outcol] = np.where(out[den] > 0, out[num] / out[den], np.nan)
    return out

def build_roster_usage(
    pb_hist: pd.DataFrame,
    team_name: str,
    roster_ids: list[int],
    last_n_games: int,
) -> pd.DataFrame:
    # only history for this team + roster
    pb_team = pb_hist[
        (pb_hist["team_name"].astype(str) == str(team_name))
        & (pb_hist["player_id"].astype(int).isin([int(x) for x in roster_ids]))
    ].copy()

    pb_team = _add_team_totals(pb_team)

    # season-ish aggregates (within history slice)
    agg = pb_team.groupby("player_id").agg(
        games_played=("game_id", "nunique"),
        avg_points=("points", "mean"),
        avg_rebounds=("reboundstotal", "mean"),
        avg_assists=("assists", "mean"),
        avg_minutes=("numminutes", "mean"),
        mean_points_share=("points_share", "mean"),
        mean_rebounds_share=("reb_share", "mean"),
        mean_assists_share=("ast_share", "mean"),
        last_seen=("game_date", "max"),
    ).reset_index()

    # last N games per player
    pb_team_sorted = pb_team.sort_values(["player_id", "game_date"])
    lastN = pb_team_sorted.groupby("player_id", group_keys=False).tail(int(last_n_games))

    last_agg = lastN.groupby("player_id").agg(
        pts_avg_lastN=("points","mean"),
        reb_avg_lastN=("reboundstotal","mean"),
        ast_avg_lastN=("assists","mean"),
        min_avg_lastN=("numminutes","mean"),
    ).reset_index()

    usage = agg.merge(last_agg, on="player_id", how="left")

    # player names (last known within this team slice; fallback to global last seen)
    names = []
    for pid in usage["player_id"].astype(int).tolist():
        rows = pb_team_sorted[pb_team_sorted["player_id"].astype(int) == int(pid)]
        if len(rows) == 0:
            rows = pb_hist[pb_hist["player_id"].astype(int) == int(pid)].sort_values("game_date")
        names.append(_safe_name_from_rows(rows, int(pid)))
    usage["player_name"] = names
    usage["team_name"] = str(team_name)

    # ensure every roster_id exists (new guys / no history)
    have = set(usage["player_id"].astype(int).tolist())
    missing = [int(x) for x in roster_ids if int(x) not in have]
    if missing:
        fb_rows = []
        for pid in missing:
            rows = pb_hist[pb_hist["player_id"].astype(int) == int(pid)].sort_values("game_date")
            fb_rows.append({
                "player_id": int(pid),
                "team_name": str(team_name),
                "player_name": _safe_name_from_rows(rows, int(pid)),
                "games_played": 0,
                "avg_points": 0.0,
                "avg_rebounds": 0.0,
                "avg_assists": 0.0,
                "avg_minutes": 0.0,
                "mean_points_share": np.nan,
                "mean_rebounds_share": np.nan,
                "mean_assists_share": np.nan,
                "pts_avg_lastN": np.nan,
                "reb_avg_lastN": np.nan,
                "ast_avg_lastN": np.nan,
                "min_avg_lastN": np.nan,
                "last_seen": pd.NaT,
            })
        usage = pd.concat([usage, pd.DataFrame(fb_rows)], ignore_index=True)

    return usage

def build_vs_opp_splits(
    pb_hist: pd.DataFrame,
    team_name: str,
    opp_team_name: str,
    roster_ids: list[int],
) -> pd.DataFrame:
    pb_vs = pb_hist[
        (pb_hist["team_name"].astype(str) == str(team_name))
        & (pb_hist["opp_team_name"].astype(str) == str(opp_team_name))
        & (pb_hist["player_id"].astype(int).isin([int(x) for x in roster_ids]))
    ].copy()

    if len(pb_vs) == 0:
        return pd.DataFrame(columns=["player_id","games_vs_opp","pts_vs_opp","reb_vs_opp","ast_vs_opp"])

    splits = pb_vs.groupby("player_id").agg(
        games_vs_opp=("game_id","nunique"),
        pts_vs_opp=("points","mean"),
        reb_vs_opp=("reboundstotal","mean"),
        ast_vs_opp=("assists","mean"),
    ).reset_index()

    return splits

# Build usage + splits for both teams
home_usage = build_roster_usage(pb_hist, home_team, home_ids, LAST_N_GAMES)
away_usage = build_roster_usage(pb_hist, away_team, away_ids, LAST_N_GAMES)

home_vs = build_vs_opp_splits(pb_hist, home_team, away_team, home_ids)
away_vs = build_vs_opp_splits(pb_hist, away_team, home_team, away_ids)

home = home_usage.merge(home_vs, on="player_id", how="left")
away = away_usage.merge(away_vs, on="player_id", how="left")

print("home usage shape:", home.shape)
display(home.sort_values("avg_points", ascending=False).head(12))

print("away usage shape:", away.shape)
display(away.sort_values("avg_points", ascending=False).head(12))


In [ ]:
# ==========================================================
# CELL 7 – BUILD PLAYER λ (POINTS/REB/AST) + SCALE TO TEAM TARGETS
# ==========================================================
def _clip(x: float, lo: float, hi: float) -> float:
    return float(min(max(x, lo), hi))

def _ratio_factor(season: float, recent: float, vsopp: float, games_vs: float) -> float:
    # recency ratio
    f_rec = 1.0
    if pd.notna(season) and season > 0 and pd.notna(recent):
        f_rec = float(recent) / float(season)

    # vs-opp ratio (shrunk toward season to avoid 1-game magic)
    f_vs = 1.0
    if pd.notna(season) and season > 0 and pd.notna(vsopp) and pd.notna(games_vs) and float(games_vs) >= float(MIN_GAMES_VS_OPP):
        g = float(games_vs)
        shrunk = (float(vsopp) * g + float(season) * float(VSOPP_SHRINK_K)) / (g + float(VSOPP_SHRINK_K))
        f_vs = float(shrunk) / float(season)

    # combine (exponent weights)
    f = (f_rec ** float(W_RECENCY)) * (f_vs ** float(W_VSOPP))
    return _clip(f, FACTOR_CLIP[0], FACTOR_CLIP[1])

def _team_points_fallback(team_name: str) -> float:
    tmp = pb_hist[pb_hist["team_name"].astype(str) == str(team_name)]
    if len(tmp) == 0:
        return float("nan")
    by_game = tmp.groupby("game_id")["points"].sum()
    return float(by_game.mean())

def _team_stat_target_from_team_boxes(team_name: str, stat: str) -> float:
    # stat in {"rebounds","assists"} mapped to likely columns
    tb = team_boxes_qepc.copy()
    if "team_name" not in tb.columns:
        return float("nan")
    tb = tb[tb["team_name"].astype(str) == str(team_name)]
    tb = tb[tb["game_date"].notna()]
    tb = tb[tb["game_date"] < asof_date]
    if LOOKBACK_DAYS is not None and LOOKBACK_DAYS > 0:
        cutoff = asof_date - timedelta(days=int(LOOKBACK_DAYS))
        tb = tb[tb["game_date"] >= cutoff]

    candidates = []
    if stat == "rebounds":
        candidates = ["reboundstotal","reboundsTotal","rebounds_total","reb_total"]
    elif stat == "assists":
        candidates = ["assists","ast","assists_total"]
    elif stat == "points":
        candidates = ["points","pts","score"]
    for c in candidates:
        if c in tb.columns:
            val = tb[c].mean()
            return float(val) if pd.notna(val) else float("nan")
    return float("nan")

def _team_stat_target_from_player_boxes(team_name: str, col: str) -> float:
    tmp = pb_hist[pb_hist["team_name"].astype(str) == str(team_name)]
    if len(tmp) == 0:
        return float("nan")
    by_game = tmp.groupby("game_id")[col].sum()
    return float(by_game.mean())

def _get_team_targets(team_name: str) -> dict:
    pts = _team_points_fallback(team_name)
    reb = _team_stat_target_from_team_boxes(team_name, "rebounds")
    ast = _team_stat_target_from_team_boxes(team_name, "assists")

    if not np.isfinite(reb):
        reb = _team_stat_target_from_player_boxes(team_name, "reboundstotal")
    if not np.isfinite(ast):
        ast = _team_stat_target_from_player_boxes(team_name, "assists")

    return {"pts": pts, "reb": reb, "ast": ast}

home_targets = _get_team_targets(home_team)
away_targets = _get_team_targets(away_team)

# If matchup engine gave point targets, use them; otherwise fall back to team averages in history slice.
if not np.isfinite(home_pts_target):
    home_pts_target = home_targets["pts"]
if not np.isfinite(away_pts_target):
    away_pts_target = away_targets["pts"]

print("Team targets (points from matchup engine if available):")
print("  home:", home_team, "| pts:", home_pts_target, "| reb:", home_targets["reb"], "| ast:", home_targets["ast"])
print("  away:", away_team, "| pts:", away_pts_target, "| reb:", away_targets["reb"], "| ast:", away_targets["ast"])

def _base_share_fallback(df: pd.DataFrame, season_col: str, share_col: str) -> pd.Series:
    s = df[share_col]
    if s.notna().sum() > 0 and float(s.fillna(0).sum()) > 0:
        return s
    # fallback to season averages
    denom = df[season_col].fillna(0).sum()
    if denom <= 0:
        return pd.Series(np.ones(len(df)) / max(len(df), 1), index=df.index)
    return df[season_col].fillna(0) / denom

def build_team_lambdas(team_df: pd.DataFrame, team_pts: float, team_reb: float, team_ast: float) -> pd.DataFrame:
    df = team_df.copy()

    # ---- POINTS ----
    if USE_SHARE_MODEL:
        base = _base_share_fallback(df, "avg_points", "mean_points_share")
        factors = df.apply(
            lambda r: _ratio_factor(
                r.get("avg_points", np.nan),
                r.get("pts_avg_lastN", np.nan),
                r.get("pts_vs_opp", np.nan),
                r.get("games_vs_opp", np.nan),
            ),
            axis=1,
        )
        adj = base.fillna(0) * factors.astype(float)
        if float(adj.sum()) <= 0:
            adj = np.ones(len(df))
        share = adj / float(adj.sum())
        df["lambda_points"] = float(team_pts) * share
    else:
        # blended per-player mean then scaled (older approach)
        w_season, w_recent, w_vs = 0.50, 0.35, 0.15
        def _blend(season, recent, vs, games_vs):
            parts, weights = [], []
            if pd.notna(season): parts.append(float(season)); weights.append(w_season)
            if pd.notna(recent): parts.append(float(recent)); weights.append(w_recent)
            if pd.notna(vs) and pd.notna(games_vs) and float(games_vs) >= float(MIN_GAMES_VS_OPP):
                parts.append(float(vs)); weights.append(w_vs)
            if not parts: return 0.0
            w = np.array(weights, dtype=float); w = w / w.sum()
            return float(np.dot(parts, w))
        df["lambda_points_raw"] = df.apply(lambda r: _blend(r.get("avg_points",np.nan), r.get("pts_avg_lastN",np.nan), r.get("pts_vs_opp",np.nan), r.get("games_vs_opp",np.nan)), axis=1)
        s = float(df["lambda_points_raw"].sum())
        df["lambda_points"] = df["lambda_points_raw"] * (float(team_pts)/s) if s > 0 else df["lambda_points_raw"]

    # ---- REBOUNDS ----
    base_reb = _base_share_fallback(df, "avg_rebounds", "mean_rebounds_share")
    reb_factors = df.apply(
        lambda r: _ratio_factor(
            r.get("avg_rebounds", np.nan),
            r.get("reb_avg_lastN", np.nan),
            r.get("reb_vs_opp", np.nan),
            r.get("games_vs_opp", np.nan),
        ),
        axis=1,
    )
    adj_reb = base_reb.fillna(0) * reb_factors.astype(float)
    if float(adj_reb.sum()) <= 0:
        adj_reb = np.ones(len(df))
    reb_share = adj_reb / float(adj_reb.sum())
    df["lambda_rebounds"] = float(team_reb) * reb_share if np.isfinite(team_reb) else df.get("avg_rebounds", 0.0)

    # ---- ASSISTS ----
    base_ast = _base_share_fallback(df, "avg_assists", "mean_assists_share")
    ast_factors = df.apply(
        lambda r: _ratio_factor(
            r.get("avg_assists", np.nan),
            r.get("ast_avg_lastN", np.nan),
            r.get("ast_vs_opp", np.nan),
            r.get("games_vs_opp", np.nan),
        ),
        axis=1,
    )
    adj_ast = base_ast.fillna(0) * ast_factors.astype(float)
    if float(adj_ast.sum()) <= 0:
        adj_ast = np.ones(len(df))
    ast_share = adj_ast / float(adj_ast.sum())
    df["lambda_assists"] = float(team_ast) * ast_share if np.isfinite(team_ast) else df.get("avg_assists", 0.0)

    return df

home_lambdas = build_team_lambdas(home, team_pts=home_pts_target, team_reb=home_targets["reb"], team_ast=home_targets["ast"])
away_lambdas = build_team_lambdas(away, team_pts=away_pts_target, team_reb=away_targets["reb"], team_ast=away_targets["ast"])

print("\nSanity checks (sums):")
print("  sum(home λ_points):", round(float(home_lambdas["lambda_points"].sum()), 3))
print("  sum(away λ_points):", round(float(away_lambdas["lambda_points"].sum()), 3))


In [ ]:
# ==========================================================
# CELL 8 – CLEAN VIEW OUTPUT + TOP PLAYERS
# ==========================================================
def clean_view(df: pd.DataFrame) -> pd.DataFrame:
    cols = [
        "team_name","player_id","player_name",
        "games_played",
        "avg_points","pts_avg_lastN","pts_vs_opp","games_vs_opp","lambda_points",
        "avg_rebounds","reb_avg_lastN","reb_vs_opp","lambda_rebounds",
        "avg_assists","ast_avg_lastN","ast_vs_opp","lambda_assists",
        "last_seen",
    ]
    keep = [c for c in cols if c in df.columns]
    out = df[keep].copy()

    # nicer rounding
    for c in [x for x in out.columns if x.startswith("avg_") or x.endswith("_lastN") or x.startswith("pts_") or x.startswith("reb_") or x.startswith("ast_") or x.startswith("lambda_")]:
        if c in out.columns and pd.api.types.is_numeric_dtype(out[c]):
            out[c] = out[c].astype(float).round(3)
    return out

home_view = clean_view(home_lambdas).sort_values("lambda_points", ascending=False).reset_index(drop=True)
away_view = clean_view(away_lambdas).sort_values("lambda_points", ascending=False).reset_index(drop=True)

print("Top home players by λ_points:")
display(home_view.head(12))

print("Top away players by λ_points:")
display(away_view.head(12))

final_view = pd.concat([home_view, away_view], ignore_index=True)
print("\nFINAL (home + away) preview:")
display(final_view.head(30))
